# Spread of infectious diseases research project

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import imageio.v2 as imageio
import os
import math

Here I created Data Structure of graph, where each node represents one person and edges represent connections between them as possible spread ways.

In [3]:
class Human:
    """
    Class representing one node in a graph
    has features in __init__
    can be infected through calling infect method
    """
    def __init__(self, resist, ill_days_const):
        """
        resist = 0 <= float <= 1 - vaccination resist
        ill_days_const - int - number of days when person is ill
        """
        self.ill = False
        self.ill_days = 0
        self.immunity_days = 0
        self.immunity = False
        self.resist = resist
        self.ill_days_const = ill_days_const
        self.dead = False
        self.had_ill = False

    def infect(self):
        """
        infect method
        checks if person is alive
        if yes than checks if person has the immunity
        and if not tries to infect, where resist is applied
        """
        if self.dead:
            return None
        if not self.ill:
            if not self.immunity:
                if random.random() > self.resist:
                    self.ill = True
                    self.had_ill = True
                    self.ill_days = self.ill_days_const
class SocietyGraph(nx.Graph):
    """
    Graph of society built on watts_strogatz_graph - small world network
    """
    def __init__(self,n,k,p, scale_free = False):
        """
        n - int - number of nodes in graph
        k - int - number of adjacent nodes for every node
        p - 0 <= float <=1 -probability of rewiring edges to non-neighbors
        scale_free - use another graph for modelling
        """
        if scale_free:
            super().__init__(nx.scale_free_graph(n))
        else:
            super().__init__(nx.watts_strogatz_graph(n,k,p))
        self.num_of_people = n
    def initialize_disease(self, infect_chance, ill_days, immunity_days, death_rate):
        """
        sets parameters to disease
        infect_chance - 0 < float <= 1, chance of one being infected if his neighbour node is ill
        ill_days - int - number of days when person is ill
        immunity_days - int - number of days when person can't be infected after beating the disease 
        """
        self.infect_chance = infect_chance
        self.ill_days = ill_days
        self.immunity_days = immunity_days
        self.death_rate = death_rate
    def initialize_society(self, vaccination_dict, big_start = False, huge_start = False,vacc_tuples = False):
        """
        Sets parameters to society
        vaccinaction_dict - {percent of people: percent of resist} -\
        describes level and percent of vaccination in society
        Additional:
        big_start - bool - three persons are infected at the beggining
        huge_start - bool - 51 persons are infected at the beggining
        vacc_tuples - List(tuple(a,b)) - if you don't want or can't use dict
        """
        self.human_list = []
        if not vacc_tuples:
            for element in vaccination_dict:
                num_of_type = round(self.num_of_people * element)
                self.human_list += [Human(vaccination_dict[element], self.ill_days) for i in range(num_of_type)]
        else:
            for element, vac in vacc_tuples:
                num_of_type = round(self.num_of_people * element)
                self.human_list += [Human(vac, self.ill_days) for i in range(num_of_type)]

        random.shuffle(self.human_list)
        for person_id in range(self.num_of_people):    
            self.human_list[person_id].neighbors = self.adj[person_id]
        self.human_list[-1].ill = True
        self.human_list[-1].ill_days = self.ill_days
        if big_start:
            self.human_list[-6].ill = True
            self.human_list[-6].ill_days = self.ill_days
            self.human_list[-12].ill = True
            self.human_list[-12].ill_days = self.ill_days
        if huge_start:
            for _ in range(100):
                nid = random.randint(0,self.num_of_people)
                self.human_list[nid].ill = True
                self.human_list[nid].ill_days = self.ill_days            
    def infect_step(self):
        """
        Simulates one day of disease
        """
        # counting people
        for person in self.human_list:
            if person.ill:
                if random.random() < 1 - (1-self.death_rate)**(1/self.ill_days):
                    person.dead = True
                    person.ill_days = 0
                    person.ill = False
                else:
                    if person.ill_days == 1:
                        person.ill_days = 0
                        person.ill = False
                        person.immunity = True
                        person.immunity_days = self.immunity_days
                    else:
                        person.ill_days -= 1
            else:
                if person.immunity:
                    if person.immunity_days == 1:
                        person.immunity = False
                        person.immunity_days = 0
                    else:
                        person.immunity_days -= 1
        # infecting
        for pos, person in enumerate(self.human_list):
            if person.ill:
                for x in person.neighbors:
                    chance = random.random()
                    #print(chance, 1 - (1-self.infect_chance)**(1/self.ill_days) )
                    if chance <= 1 - (1-self.infect_chance)**(1/self.ill_days):
                        self.human_list[x].infect()

    def color_graph(self):
        """
        Makes color_map parameter for gif to have colors
        """
        self.color_map = []
        for node in self:
            if self.human_list[node].dead == True:
                self.color_map.append('black')               
            elif self.human_list[node].ill == True:
                self.color_map.append('red')
            elif self.human_list[node].immunity == True:
                self.color_map.append('green')
            else:
                self.color_map.append('blue')
    def make_a_gif(self, epochs):
        """
        creates a gif for period of epochs
        epochs - int - number of days you want to have in you gif
        """
        pos = nx.spring_layout(b)
        for i in range(1000,1000+epochs):
            if i!=0:
                b.infect_step()
            b.color_graph()
            fig = plt.figure(figsize=(20,20))
            nx.draw(b, node_color = b.color_map, pos=pos)
            custom_node_attrs = {}
            for node in b.nodes:
                custom_node_attrs[node] = b.human_list[node].resist
            nx.draw_networkx_labels(b,pos,custom_node_attrs)
            fig.savefig(os.getcwd() + f'/images_graph/img_{i}.png')
            plt.close()
        path = os.getcwd()
        png_dir = path +'/images_graph'
        png_files = sorted([os.path.join(png_dir, f) for f in os.listdir(png_dir)])
        i = len(os.listdir(path + '/graphs_gif'))
        gif_writer = imageio.get_writer(path+f'/graphs_gif/animation_{i}.gif', mode='I', duration=500)
        for png_file in png_files[:epochs]:
            image = imageio.imread(png_file)
            gif_writer.append_data(image)
        gif_writer.close()
    def make_a_plot(self, epochs, quarantine_threshold = 999999 , quarantine_strength = 1, q_time = 30):
        """
        makes 2 plots - one with dead and ill, one with had been ill persons
        You can make a quarantine after certain threshold
        Parameters
        epochs - number of days you want to simulate
        Additional:
        quarantine_threshold - if this number of people is ill quarantine becomes active
        quarantine_strength - in this many times infection chance will decrease
        q_time - quarantine time
        """
        list_of_ill_time = []
        list_of_had_ill_time = []
        list_of_dead_time = []
        list_of_new_cases = []
        self.CHANCE = self.infect_chance
        time = [i for i in range(epochs)]
        q_now = 0
        for i in range(epochs):
            self.infect_step()
            ill = 0
            dead = 0 
            had_ill = 0
            ill_new = 0
            for elem in self.human_list:
                if elem.had_ill:
                    had_ill+= 1
                if elem.dead:
                    dead += 1
                if elem.ill:
                    ill += 1
                if elem.ill_days == self.ill_days:
                    ill_new += 1
            list_of_new_cases.append(ill_new)
            list_of_ill_time.append(ill)
            list_of_had_ill_time.append(had_ill)
            list_of_dead_time.append(dead)
            if ill >= quarantine_threshold and q_now == 0:
                self.infect_chance = self.CHANCE / quarantine_strength
                q_now = q_time
            if q_now != 0:
                q_now = q_now -1
                if q_now == 0:
                    self.infect_chance = self.CHANCE
        plt.figure()
        plt.title('Number of people currently ill')
        plt.xlabel('Time')
        plt.ylabel('Number of people')
        plt.plot(time, list_of_ill_time, '-')
        plt.figure()
        plt.title('Number of new cases')
        plt.xlabel('Time')
        plt.ylabel('Number of people')
        plt.plot(time, list_of_new_cases, '-')
        plt.figure()
        plt.title('Number of people dead')
        plt.xlabel('Time')
        plt.ylabel('Number of people')
        plt.plot(time, list_of_dead_time, '-')
        plt.figure()
        plt.title('Number of people who have ever been ill')
        plt.xlabel('Time')
        plt.ylabel('Number of people')
        plt.plot(time, list_of_had_ill_time, '-')


#####  To model the society, I used the small-world network graph, namely watts-strogatz from the NetworX library, to which you should pass the number of neighbors and how random the connections will be as a parameter. For understanding, I suggest viewing the image that visualizes the small-world network graph.

![text](final_plots/photo.jpg)


###  GIF
Initially, all people are colored blue.  We use black to depict a person who has died of an illness, red for a person who is sick, and green for a person with immunity.


![](final_plots/animation_84.gif)

### Plots
There are 4 plots for each run of infection.
- Number of ill people at the moment
- Number of new cases per day
- Number of dead people
- Number of people who have ever been ill

### Research on the contagiousness of a new virus on society

First of all, we need to initialize the parameters:
- 1. All subsequent experiments will be conducted assuming that we are modeling a city of 500 thousand people. The number of simulation days is 800.
- 2. In this part, we model the society as a small-world network with 6 neighbors and a 30% chance of an edge being intertwined (approximately 4 close people and 2 distant ones)
- 3. We take data for infections based on covid, but its parameters changed significantly from strain to strain, so our data are estimates.
- 4. For the disease, we set the number of days of illness as 12 and the number of days of immunity as 180
- 5. The purpose of the study will be to determine the dependence of the number of patients on changes in the infectivity value, and thus the "reproduction number" (or how many people one person infects)

According to the article, which refers to the WHO, the "reproduction rate" (or how many people one person infects) for covid ranges from 1.5 to 2.4
Results of the simulation for such values:
-  1.62

![](final_plots/res_1_027_a.png)
![](final_plots/res_1_027_d.png)
- 1.74

![](final_plots/res_1_029_a.png)
![](final_plots/res_1_029_d.png)
-  1.86

![](final_plots/res_1_031_a.png)
![](final_plots/res_1_031_d.png)
-  1.98

![](final_plots/res_1_033_a.png)
![](final_plots/res_1_033_d.png)
-  3.6 - just to look at results when number is way too big

![](final_plots/res_1_060_a.png)
![](final_plots/res_1_060_d.png)

As we can see from 1.5, we have two distinct peaks. Since there is no vaccine or quarantine, a large number of people get sick, and as the virus circulates, it reaches the people who were the first to get sick and no longer have immunity and re-infects them. This is how we get the second wave, as shown in the first three graphs. It should be noted that the number of people who have been infected since the first wave is growing - 250, 300, 350. It is the same with the second wave.

We see a different pattern at a contagion rate of 1.98. About 400,000 people fell ill during the first wave. Accordingly, almost 80% of the population was immune at the same time, so the second wave did not occur.

We also modeled an extreme case in the form of 3.6 - as we can see, almost all people get sick at once, so there can be no second wave as such - the virus disappears.

### Study of the impact of vaccination on the spread of the virus in society

Now let's consider vaccination against the virus in humans:
From the parameters we take:

- Population - 500.000
- Reproduction rate - 2.28
- Number of neighbors - 6
- Chance of changing edges - 0.5

There are two types of vaccines - efficiency 0.8 and 0.6

Firstly let's take at look at efficiency 0.8:
- 20% of society are vaccinated

![](final_plots/res_2_820_a.png)
![](final_plots/res_2_820_d.png)
- 40% of society are vaccinated

![](final_plots/res_2_840_a.png)
![](final_plots/res_2_840_d.png)
- 60% of society are vaccinated

![](final_plots/res_2_860_a.png)
![](final_plots/res_2_860_d.png)
- 80% of society are vaccinated

![](final_plots/res_2_880_a.png)
![](final_plots/res_2_880_d.png)

Now 0.6 
- 40% of society are vaccinated

![](final_plots/res_2_640_a.png)
![](final_plots/res_2_640_d.png)

- 80% of society are vaccinated

![](final_plots/res_2_680_a.png)
![](final_plots/res_2_680_d.png)

- 95% of society are vaccinated

![](final_plots/res_2_695_a.png)
![](final_plots/res_2_695_d.png)


### Study of the structure of society on the spread of the virus

Here I experemented with different parameters for graph:
- k = 6 , p = 0.2

![](final_plots/res_3_1.png)
![](final_plots/res_3_2.png)

- k = 6, p = 0.5

![](final_plots/res_3_3.png)
![](final_plots/res_3_4.png)

- k = 6, p = 0.25

![](final_plots/res_3_9.png)
![](final_plots/res_3_10.png)

- k = 10, p = 0.55

![](final_plots/res_3_5.png)
![](final_plots/res_3_6.png)

- k = 15 , p = 0.7

![](final_plots/res_3_7.png)
![](final_plots/res_3_8.png)

Note: for all graphs, the reproduction rate was ~1.8

From the first two graphs, we can conclude that with more contacts with relatively distant people, the number of cases in the first wave increases significantly. Although no second wave was formed in the second situation because of this effect, the situation would be worse than in the first case if new patients entered the city.

The next three graphs show a rather unexpected result. For these parameters, we have 4.5 close people and 1.5, 5.5, and 10.5 distant people, respectively. If we keep the reproduction rate constant, we get that fewer people with more distant ties were infected. And based on the fact that we know the coefficient, with good statistical data, in theory we could better find out whether there were more infections from close or distant contacts, but unfortunately, we cannot do this for sure yet.

### Study of the effectiveness of various quarantine measures

Parameters
- Population - 500.000
- Number of reproductions - 1.8
- Number of neighbors - 6
- Chance of changing edges - 0.3

Without quarantine:

 ![](final_plots/res_4_4_1.png)
 ![](final_plots/res_4_4_2.png)

Here I designed 3 scenarios:
 - 1. After reaching 2000 (0.4%) simultaneous patients, a weak quarantine is introduced for 30 days (reproduction decreases by a factor of 1.1)

 ![](final_plots/res_4_1_1.png)
 ![](final_plots/res_4_1_2.png) 
 - 2. After 5000 (1%) simultaneous patients, an average quarantine of 30 days is introduced (reproduction decreases in 1.2)

 ![](final_plots/res_4_2_1.png)
 ![](final_plots/res_4_2_2.png) 
 - 3. After 10,000 (2%) simultaneous cases, a strong quarantine is introduced for 30 days (reproduction decreases by a factor of 1.3)
 
 ![](final_plots/res_4_3_1.png)
 ![](final_plots/res_4_3_2.png)

As you can see, different types of quarantines had different effects:
 - Weak quarantine, although it did not stop the spread and, as can be seen from the graph, there will be a second wave, still had an impact.
 - The medium quarantine was able to significantly reduce the number of patients, but keeps the number of patients at the quarantine level and a second wave is possible.
 - A strong quarantine cannot be taken as the only measure of protection, because the number of patients will simply rise and fall relative to a given limit.

Source plots -  https://colab.research.google.com/drive/1s9W1AUTVftCKyqrNmrhgCQSSZ8mQkHMQ?usp=sharing

# RESULTS

1) Even a small change in the infectivity of the virus has a significant impact on the number of cases. At certain average infectivity values, the virus can receive new waves of infections even within a city of 500,000 people. At high infectivity, a larger outbreak occurs, which is not followed by subsequent waves.
2) The vaccine matters, both in quantity and quality. For a vaccine with 80% effectiveness, we see that with ~70% of the population vaccinated, we get collective immunity. For a 60% effective vaccine, this number is closer to ~90%.
3) Much depends on the graph model we choose. Moreover, a lot depends on the parameters of the graph. In particular, we were able to see an interesting dependence on the number of long-distance connections in people.
4) Different quarantines have different effects, but obviously, you can't allow a large number of patients, otherwise you will have to introduce a very long quarantine to fix the situation. It is better to quarantine from the very beginning of the disease and increase its levels if necessary.

##### So, it is possible to simulate a society using graphs. To do this, you need to choose the right graph structure that best suits your situation (this can be a small-world network). By realizing each node as a person, we can achieve plausible results, such as 70% of those vaccinated for collective immunity. Depending on the situation, you can add restrictions that will work, such as quarantine.их для колективного імунітету. В залежності від ситуації можна добавляти обмеження, які працюватимуть - наприклад, карантин.